In [1]:
import numpy as np
import pandas as pd

print("numpy  version: ", np.__version__)
print("pandas version: ", pd.__version__)

numpy  version:  1.23.5
pandas version:  2.1.4


# Часть 3. Операции над данными

Работая с датафреймами, нужно помнить, что они реализованы с использованием массивов NumPy. Это автоматически даёт возможность применять подходы, которые справедливы для последних.

При этом нужно помнить про следующие аспекты:
- при использовании унарных операций будет сохранён индекс и название столбца, к которому операция была применена
- в случае бинарных операций Pandas автоматически сам совместит индексы для совместности операндов (именно поэтому нужно быть внимательным во время работы с данными из разных источников)

Посмотрим, как сохраняется индексация при использовании UFuncs:

In [2]:
df = pd.DataFrame(
    np.random.randint(0, 10, (3, 4)),
    columns=["A", "B", "C", "D"]
)

df

,A,B,C,D
0,3,5,7,1
1,0,1,0,9
2,6,6,9,6


In [3]:
# мы можем применить универсальную функцию ко всему датафрейму сразу
np.sin(df * np.pi / 4)

,A,B,C,D
0,0.707107,-0.707107,-0.707107,0.707107
1,0.000000,0.707107,0.000000,0.707107
2,-1.000000,-1.000000,0.707107,-1.000000


In [4]:
# проверим, как работает выравнивание индексов для произвольных Series
# обратите внимание, что индексы объектов не полностью идентичны
s1 = pd.Series({"a": 11, "b": 2, "c": 3})
s2 = pd.Series({"b": 20, "c": 30, "d": 44})

s1 + s2

a     NaN
b    22.0
c    33.0
d     NaN
dtype: float64

Видно, что для индексов, которые присутствуют лишь в одном объекте, операцию выполнить не удалось (что соответствует здравому смыслу). Посмотрим на результаты объединения и пересечения индексов для наглядности:

In [5]:
print("Объединение индексов: ", s1.index.union(s2.index))
print("Пересечение индексов: ", s1.index.intersection(s2.index))

Объединение индексов:  Index(['a', 'b', 'c', 'd'], dtype='object')
Пересечение индексов:  Index(['b', 'c'], dtype='object')


Как видно, во время применения бинарного оператора произошло совмещение индексов и соответствующая операция была применена для элементов, которые находятся на позициях соответствующих индексов.

Если же хочется избавиться от NaN'ов, то нужно помочь Pandas'у. Для этого есть возможность указать специальный аргумент `fill_value`, который будет использоваться в случае отсутствующего ключа.

In [6]:
# заполним отсутствующие значения нулями
s1.add(s2, fill_value=0.0)

a    11.0
b    22.0
c    33.0
d    44.0
dtype: float64

Применение универсальных функций к датафреймам подчиняется абсолютно тем же правилам, которые были упомянуты для Series:

In [7]:
df_1 = pd.DataFrame(
    np.random.randint(1, 11, (2, 2)),
    columns=["a", "b"]
)

df_1

,a,b
0,9,8
1,5,5


In [8]:
df_2 = pd.DataFrame(
    np.random.randint(1, 11, (3, 3)),
    columns=["a", "b", "c"]
)

df_2

,a,b,c
0,10,1,3
1,10,2,5
2,10,6,2


По аналогии с Series будет проводиться совмещение индексов и столбцов. Если по итогам смещения один из операндов будет отсутствовать, то мы увидим NaN (которые можно будет устранить тем же самым образом):

In [60]:
df_1 + df_2

,a,b,c
0,13.0,13.0,NaN
1,15.0,15.0,NaN
2,NaN,NaN,NaN


In [61]:
# уберём NaN'ы
df_1.add(df_2, fill_value=0.0)

,a,b,c
0,13.0,13.0,7.0
1,15.0,15.0,2.0
2,10.0,6.0,10.0


Мы рассмотрели, как можно производить операции между Series и DataFrame. Однако в Pandas можно осуществлять операции и между объектами разных типов. Давайте посмотрим, как будут выполняться такие операции:

In [9]:
df = pd.DataFrame(
    np.random.randint(1, 11, (3, 4)),
    columns=["c1", "c2", "c3", "c4"]
)

df

,c1,c2,c3,c4
0,8,8,8,7
1,10,5,3,3
2,10,5,7,1


По умолчанию операции производятся построчно:

In [10]:
s = pd.Series({"c1": 1, "c2": 2, "c3": 3, "c4": 4})
df - s

,c1,c2,c3,c4
0,7,6,5,3
1,9,3,0,-1
2,9,3,4,-3


Если же операцию нужно выполнить по столбцам, то нужно об этом явно сказать Pandas'у с помощью аргумента `axis`:

In [11]:
c = np.array([1, 2, 3])

df.subtract(c, axis=0)

,c1,c2,c3,c4
0,7,7,7,6
1,8,3,1,1
2,7,2,4,-2


# Часть 4.  Обработка отсутствующих значений

#  Общие понятия

Ранее мы столкнулись не только с кейсом отсутствующих данных (NaN'ы, которые появляются при операциях над объектами с отличающимися индексами), но и способами их обработки. В этой части урока мы чуть подробнее остановимся на этой теме.

Прежде всего, следует отметить, что любой способ взаимодействия с отсутствующими (или ошибочными) данными имеет как свои достоинства, так и недостатки. Кроме того, даже наличие данных порой может быть индикатором того, что на самом деле в данных имеется проблема (например, если мы знаем, что в поле при нормально сценарии функционирования должны быть положительные целые значения, но мы вдруг встречаем `-999`)

Именно по этой причине при работе с данными критически важно понимать домен, в которым вы сейчас работаете (не только с точки зрения бизнеса, но и с точки зрения технологий).

Как мы часто видели в различных кейсах ранее, использование нативных инструментов Python зачастую медленнее специализированных аналогов. В случае с обработкой отсутствующих значений тоже есть важный нюанс, который заключается в разнице между `None` и `np.nan`. Для этого сравним, какой `dtype` проставляется в случае использования каждого из них:

In [12]:
native_missing = np.array([1, 2, None, 4])
numpy_missing = np.array([1, 2, np.nan, 4])

print("Native dtype: ", native_missing.dtype)
print("NumPy  dtype: ", numpy_missing.dtype)

Native dtype:  object
NumPy  dtype:  float64


Обратите внимание, что при использовании `None` проставляется тип `object`, которые требует больше времени на обработку:

In [13]:
%timeit np.arange(25_000_000, dtype=object).sum()
%timeit np.arange(25_000_000, dtype=float).sum()

1.71 s ± 377 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
99.3 ms ± 8.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Как видно, использование именно числового типа позволяет производить обработку эффективнее. Именно по этой причине будьте внимательны с теми объектами, с которыми вы работаете.

Кроме того, обратите внимание, что `np.nan` является числом с плавающей точкой. Привести его к целочисленному типу нельзя:

In [15]:
try:
    np.array([1, 2, np.nan], dtype=int)
except Exception as e:
    print("ERROR:\n", e)

ERROR:
 cannot convert float NaN to integer


Также не стоит забывать особенности взаимодействия других числе с `pd.nan`: в случае с любой операцией даже единичное его присутствие приводит к тому, что весь производный объект становится тем же значением:

In [16]:
x = np.arange(1_000, dtype=float)
x[-1] = np.nan
x.sum()

nan

Чтобы избежать этой проблемы у NumPy есть NaN-safe эквиваленты соответствующих операций:

In [17]:
print("NaN-safe sum: ", np.nansum(x))
print("NaN-safe max: ", np.nanmax(x))
print("NaN-safe min: ", np.nanmin(x))

NaN-safe sum:  498501.0
NaN-safe max:  998.0
NaN-safe min:  0.0


Работать с пропущенными значениями в Pandas даже удобнее по одной простой причине: он самостоятельно при наличии возможности обеспечивает конвертацию `None -> np.nan`

In [18]:
# Pandas автоматически конвертирует None => NaN
# также автоматически производится конвертация во float
pd.Series([1, None, 2, np.nan])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

# Операции над пропущенными значениями

Посмотрим, что можно делать с пропущенными значениями в Pandas:

In [19]:
s = pd.Series([1, None, 2, np.nan])

In [20]:
# проверка на пустоту (возвращает маску)
s.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [21]:
# данную маску удобно использовать, чтобы вытащить соответствующие непустые элементы
s[s.notnull()]

0    1.0
2    2.0
dtype: float64

In [22]:
# удалить пропущенные значения
s.dropna()

0    1.0
2    2.0
dtype: float64

In [23]:
# заполнить пропуски значением
s.fillna(-999)

0      1.0
1   -999.0
2      2.0
3   -999.0
dtype: float64

При заполнении пропусков есть удобные функции, которые используют предшествующие значения. Будьте осторожны, потому что в некоторых случаях всё-таки могут остаться пустующие значения:

In [25]:
# forward-fill
s.ffill()

0    1.0
1    1.0
2    2.0
3    2.0
dtype: float64

In [26]:
# back-fill
s.bfill()

0    1.0
1    2.0
2    2.0
3    NaN
dtype: float64

Нам осталось посмотреть, как эти же функции работают в случае датафреймов:

In [27]:
df = pd.DataFrame(
    np.arange(1, 13).reshape((3, 4)),
    columns=["a", "b", "c", "d"]
)
df.loc[1, "a":"b"] = np.nan
df.loc[2, "d"] = np.nan

df

,a,b,c,d
0,1.0,2.0,3,4.0
1,NaN,NaN,7,8.0
2,9.0,10.0,11,NaN


In [28]:
# проверка на пустоту
df.isnull()

,a,b,c,d
0,False,False,False,False
1,True,True,False,False
2,False,False,False,True


In [29]:
# удалить пустые значения 
# (обратите внимание, что по умолчанию происходит работа в построчном режиме)
df.dropna()

,a,b,c,d
0,1.0,2.0,3,4.0


In [30]:
# удалить пустые значения можно и в режиме работы по столбцам 
# (для этого нужно задать аргумент axis)
df.dropna(axis=1)

,c
0,3
1,7
2,11


Отметим, что режимы работы по каждом из измерений также доступны и для всех упомянутых ранее способов обработки (включая `fillna`, `ffill`, `bfill`)